# Run points THDM-II model

The idea is run the model

In [1]:
import numpy as np
import pandas as pd
import subprocess
import time

%matplotlib inline
import matplotlib.pyplot as plt 

Input file:

In [3]:
xd0 = pd.read_csv('L4_Negative_tb_2073.txt')

In [7]:
xd0[:2]

,L1,L2,L3,L4,L5,M12,tb
0,0.134761,0.2956,0.041732,-0.08504,-0.345209,-70794.6,2.07321
1,0.134761,0.2956,0.041732,-0.08504,-0.345209,-72443.6,2.07321


In [32]:
import pyslha
import pyTHDM_LesHouches_generator
import sys
import time

In [33]:
# Tiempo inicial
t1=time.time() 

x=[]

#Change
Num = 10
#Num = xd0.shape[0]

for i in range (0,Num):
       
    L1 = xd0['L1'][i] 
    L2 = xd0['L2'][i]
    L3 = xd0['L3'][i]
    L4 = xd0['L4'][i]
    L5 = xd0['L5'][i]
    M12 = xd0['M12'][i]
    tb = 2.07321

    #Open the file that will created the LesHouches file
    xdict = pyTHDM_LesHouches_generator.buildSLHAinFile()

    xdict.blocks['MINPAR'].entries[1]='%.6E    # Lambda1Input'%L1
    xdict.blocks['MINPAR'].entries[2]='%.6E    # Lambda2Input'%L2
    xdict.blocks['MINPAR'].entries[3]='%.6E    # Lambda3Input'%L3
    xdict.blocks['MINPAR'].entries[4]='%.6E    # Lambda4Input'%L4
    xdict.blocks['MINPAR'].entries[5]='%.6E    # Lambda5Input'%L5
    xdict.blocks['MINPAR'].entries[9]='%.6E    # M12inputt'%M12
    xdict.blocks['MINPAR'].entries[10]='%.6E    # TanBeta' %tb

    #Write and created the Leshouches file generator and closed
    pyslha.writeSLHAFile('LesHouches.in.THDMII_low',xdict)  

    #run SPheno
    spheno = subprocess.getoutput('~/Work/SPheno-4.0.4/bin/SPhenoTHDMII LesHouches.in.THDMII_low')    
    SPheno_output = subprocess.getoutput('cat SPheno.spc.THDMII')
    
    #capturo algo de spheno
    #MASAS
    mh1 = eval(SPheno_output.split('#   PDG code')[1].split()[3])
    mh2 = eval(SPheno_output.split('#   PDG code')[1].split()[7])
    mAh = eval(SPheno_output.split('#   PDG code')[1].split()[11])
    mHm = eval(SPheno_output.split('#   PDG code')[1].split()[15])
    #print("========== MASAS ==============")
    #print("mh1=",mh1,"mh2=",mh2,"mAh=",mAh,"mHm=",mHm)
    #print("===============================\n")

    #Couplings
    h1bb = eval(SPheno_output.split('HiggsCouplingsFermions')[1].split()[1])
    h1tt = eval(SPheno_output.split('# h_1 s s coupling')[1].split()[0])
    h1tautau = eval(SPheno_output.split('# h_1 c c coupling')[1].split()[0])
    h1VV = eval(SPheno_output.split('HiggsCouplingsBosons')[1].split()[1])
    
    #print("============ Couplings ================")
    #print("chbb=",h1bb,"chtt=",h1tt,"chtautau=",h1tautau,"hVV=",h1VV)
    #print("============================\n")
    
    #RUN Higgsbounds
    
    #copio los archivos a la carpeta Input_files
    subprocess.getoutput('cp Messages.out effC.dat BR* Key.dat LesHouches.in.THDMII_low MH* SPheno.spc.THDMII Input_Files/')
    #run HiggsBounds
    hb = subprocess.getoutput('~/Work/HiggsBounds-5.3.2beta/./HiggsBounds LandH effC 3 1 ~/Work/Documents_compartidos/THDM-II/Input_Files/')
    
    #Capturo lo que necesito del archivo de resultados
    HB_results = subprocess.getoutput('cat Input_Files/HiggsBounds_results.dat')
    HBresult = eval(HB_results.split('cols:')[1].split()[15])
    #print("HBresult=",HBresult)
    
    x.append([L1,L2,L3,L4,L5,M12,tb,mh1,mh2,mAh,mHm,h1bb,h1tt,h1tautau,h1VV,HBresult])

x=np.asarray(x)
#Write the output
xd=pd.DataFrame(x,columns=['L1','L2','L3','L4','L5','M12','tb','mh1','mh2','mAh','mHm','h1bb','h1tt','h1tautau','h1VV','HBresult'])  

t2=time.time() 
print ("The program spent", t2-t1, "s running",Num,"times")   

The program spent 18.958154916763306 s running 10 times


In [38]:
xd0.shape[0]*2/60

14.7

In [39]:
xd

,L1,L2,L3,L4,L5,M12,tb,mh1,mh2,mAh,mHm,h1bb,h1tt,h1tautau,h1VV,HBresult
0,0.134761,0.295600,0.041732,-0.085040,-0.345209,-70794.6,2.07321,124.723339,444.436663,449.274477,440.410308,1.178093,0.953809,1.178093,0.996141,1.0
1,0.134761,0.295600,0.041732,-0.085040,-0.345209,-72443.6,2.07321,124.850473,449.117275,453.940155,445.168898,1.174176,0.954932,1.174176,0.996313,1.0
2,0.134761,0.295600,0.041732,-0.085040,-0.345209,-74131.0,2.07321,124.974660,453.858505,458.665353,449.986210,1.170341,0.956027,1.170341,0.996477,1.0
3,0.134761,0.295600,0.041732,-0.085040,-0.345209,-75857.8,2.07321,125.095973,458.661244,463.451007,454.863188,1.166585,0.957094,1.166585,0.996634,1.0
4,0.134761,0.295600,0.041732,-0.085040,-0.345209,-77624.7,2.07321,125.214464,463.525495,468.297175,459.799878,1.162908,0.958135,1.162908,0.996785,1.0
5,0.134761,0.295600,0.041732,-0.085040,-0.345209,-79432.8,2.07321,125.330208,468.452327,473.204967,464.797401,1.159308,0.959150,1.159308,0.996928,1.0
6,0.134761,0.295600,0.041732,-0.085040,-0.345209,-81283.1,2.07321,125.443270,473.442468,478.175158,469.856537,1.155783,0.960139,1.155783,0.997065,1.0
7,0.134761,0.295600,0.041732,-0.085040,-0.345209,-83176.4,2.07321,125.553701,478.496064,483.207939,474.977468,1.152333,0.961103,1.152333,0.997196,1.0
8,0.134844,0.295635,0.041776,-0.084092,-0.344251,-67608.3,2.07321,124.636197,435.210591,440.053084,430.999656,1.185937,0.951545,1.185937,0.995785,1.0
9,0.134844,0.295635,0.041776,-0.084092,-0.344251,-69183.1,2.07321,124.768894,439.772270,444.602297,435.643420,1.181857,0.952725,1.181857,0.995972,1.0


save output file:

In [40]:
xd.to_csv('L4_Negative_tb_2073_output.csv')